In [1]:
import sys
sys.path.append('..')

import math
import numpy as np

import metrics

# Normal distribution (Gaussian)

$$X \sim \mathcal{N}(\mu, \sigma^2)$$
Parameters:
- $\mu$: mean
- $\sigma^2 \geq 0$: variance


$$\text{PDF: } f(x) = \frac{1}{\sqrt{2\pi \sigma^2}} \text{exp}(-\frac{(x - \mu)^2}{2\sigma^2})$$

$$\text{CDF: } F(x) = \frac{1}{2}[1 + \text{erf}(\frac{x - \mu}{\sigma \sqrt{2}})]$$
$$\mathbb{E}[X] = \mu$$
$$\text{Var}(X) = \sigma^2$$

$$ \text{erf}(x) = \frac{1}{\sqrt{\pi}} \int_{-x}^{x} e^{-t^2}dt$$

In [2]:
_box_muller = [None]
def norm_box_muller():    
    if _box_muller[0] is not None:
        res = _box_muller[0]
        _box_muller[0] = None
        return res
    
    u1, u2 = np.random.rand(2)
    r = np.sqrt(-2*np.log(u1))
    theta = 2*np.pi*u2
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    _box_muller[0] = x
    return y

_marsagalia_polar = [None]
def norm_marsagalia_polar():
    if _marsagalia_polar[0] is not None:
        res = _marsagalia_polar[0]
        _marsagalia_polar[0] = None
        return res

    while True:
        x, y = 2 * np.random.rand(2) - 1
        s = x**2 + y**2
        if s < 1 and s>0:
            break
    
    f = np.sqrt((-2*np.log(s))/s)
    a, b = x*f, y*f
    _marsagalia_polar[0] = a
    return b
    
    

N = 1000000

x = np.random.randn(N) * 4.5 - 1.3
print('[NP]  mu =', np.mean(x))
print('[NP] std =', np.std(x))


x = np.empty(N)
for i in range(N): x[i] = 4.5 * norm_box_muller() - 1.3
print('[BM]  mu =', np.mean(x))
print('[BM] std =', np.std(x))

x = np.empty(N)
for i in range(N): x[i] = 4.5 * norm_marsagalia_polar() - 1.3
print('[MP]  mu =', np.mean(x))
print('[MP] std =', np.std(x))

[NP]  mu = -1.3060774730949005
[NP] std = 4.501349077009439
[BM]  mu = -1.2938543196406362
[BM] std = 4.500840021245095
[MP]  mu = -1.304276897041245
[MP] std = 4.502686854824234


In [3]:
#Generate from gaussian using quantile function
import scipy.stats


def norm_cdf(x):
    return 1/2 * (1 + scipy.special.erf(x / np.sqrt(2)))

def norm_quantile(x):
    def f(v):
        return norm_cdf(v) - x
    return scipy.optimize.brentq(f, -10, 10)

def randn_qt(size):
    u = np.random.rand(size)
    x = np.array([norm_quantile(v) for v in u])
    return x

v = 0.6
b1 = scipy.stats.norm.ppf(v)
b2 = norm_quantile(v)
print(b1)
print(b2)
print(metrics.tdist(b1, b2))





x = randn_qt(100000) * 4.5 - 1.3
print('[QT]  mu =', np.mean(x))
print('[QT] std =', np.std(x))

0.2533471031357997
0.2533471031357997
0.0
[QT]  mu = -1.307550832384208
[QT] std = 4.497997376412496


# Binomial distribution

$$X \sim B(n, p)$$
Parameters:
- $n$: number of trials
- $p \in [0, 1]$: success probability for each trial.

$p(X = k)$: $k$: number of successes.

$$\text{PMF: } f(k) = \binom{n}{k} p^k(1-p)^{n-k}$$
$$\mathbb{E}[X] = np$$
$$\text{Var}(X) = np(1 - p)$$

$$\binom{n}{k} = \frac{n!}{k!(n-k)!}$$

# Multinomial distribution

Parameters:
- $n$: number of trials
- $p_i$: probability of event $i$: $\sum p_i = 1$, $p_i >= 0$

$X$ discrete vector of size $K$: $X_i$: number of realisations of the event $i$.

$$\text{PMF: } f(x) = \binom{n}{x_1\text{...} x_k} \prod_{i=1}^K p_i^{x_i}$$

$$\mathbb{E}[X_i] = np_i$$
$$\text{Var}(X_i) = np_i(1 - p_i)$$
$$\text{Cov}(X_i, X_j) = -np_ip_j \space (i \neq j)$$

$$\binom{n}{k_1 \text{...} k_m}= \frac{n!}{\prod_{i=1}^m k_i!}$$

In [4]:
def rand_multinomial(p):
    s = 0
    p2 = np.empty(len(p))
    for i in range(len(p)-1):
        s += p[i]
        p2[i] = s
    p2[-1] = 1
    
    u = np.random.rand()
    k = 0
    while u > p2[k]:
        k += 1
    return k

N = 1000000
x = np.empty(N).astype(np.int)
p = [0.1, 0.6, 0.3]
for i in range(N):
    x[i] = rand_multinomial(p)
    
print('p[0]:', np.mean(x==0))
print('p[1]:', np.mean(x==1))
print('p[2]:', np.mean(x==2))

p[0]: 0.099871
p[1]: 0.600026
p[2]: 0.300103


# Multivariate Normal distribution

$$X \sim \mathcal{N}(\mu, \Sigma)$$
Parameters:
- $\mu \in \mathbb{R}^p$: mean
- $\Sigma \in \mathbb{R}^{p*p}$: covariance matrix (positive semi-definite)

$$\text{PDF: } f(x) = ((2\pi)^{p} \text{det}(\Sigma))^{-\frac{1}{2}} \exp(-\frac{1}{2} (x - \mu)^T \Sigma^{-1}(x-\mu))$$

$$\mathbb{E}[X] = \mu$$
$$\text{Var}(X) = \Sigma$$

In [5]:

rmu = np.array([0.5, -1.2, 4.6])
rsig = np.array([[0.4, 1.2, -1.8],[2.5,-2.8,-1.9],[-1.4,6.7,2.5]])
rsig = rsig.T @ rsig
N = 1000000

print('mu =', rmu)
print('sig=')
print(rsig)

X = np.random.multivariate_normal(rmu, rsig, size=N, check_valid='raise')
mu = np.mean(X, axis=0)
sig = 1/N * (X - mu.reshape(1,3)).T @ (X - mu.reshape(1,3))
print('[NP] mu =', mu)
print('[NP] sig=')
print(sig)


def normal_multivariate(mu, sig, size):
    N = size
    p = len(mu)
    X = np.empty((N,p))
    d, V = np.linalg.eig(sig)
    Q = np.sqrt(d).reshape(1,p) * V 
    
    
    for i in range(N):
        xn = np.random.randn(p)
        X[i] = Q @ xn + mu
    return X
    
X = normal_multivariate(rmu, rsig, size=N)
mu = np.mean(X, axis=0)
sig = 1/N * (X - mu.reshape(1,3)).T @ (X - mu.reshape(1,3))
print('mu =', mu)
print('sig=')
print(sig)

mu = [ 0.5 -1.2  4.6]
sig=
[[  8.37 -15.9   -8.97]
 [-15.9   54.17  19.91]
 [ -8.97  19.91  13.1 ]]
[NP] mu = [ 0.50043231 -1.21008451  4.59870724]
[NP] sig=
[[  8.36416271 -15.87348513  -8.95445873]
 [-15.87348513  54.11110715  19.8626531 ]
 [ -8.95445873  19.8626531   13.07519114]]
mu = [ 0.50163061 -1.20296257  4.59729319]
sig=
[[  8.37659861 -15.91528291  -8.97483976]
 [-15.91528291  54.14733662  19.92043082]
 [ -8.97483976  19.92043082  13.09552772]]


# Beta distribution

$$X \sim \text{Beta}(\alpha, \beta)$$

Parameters:
- $\alpha \in \mathbb{R} > 0$
- $\beta \in \mathbb{R} > 0$

The parameter $x \in \mathbb{R}$ must bet in $[0,1]$

$$\text{PDF: } f(x) = \frac{x^{\alpha-1} (1-x)^{\beta - 1}}{B(\alpha,\beta)}$$

$$\text{where } B(\alpha,\beta) = \frac{\Gamma (\alpha) \Gamma(\beta)}{\Gamma (\alpha + \beta)}$$

$$\text{where } \Gamma(z) = \int_{0}^{+\infty} x^{z-1} e^{-x}dx$$  

$$E[X] = \frac{\alpha}{\alpha + \beta}$$
$$\text{Var}(X) = \frac{\alpha\beta}{(\alpha+\beta)^2(\alpha+\beta+1)}$$

The beta distribution is the conjugate prior probability distribution of the bernoulli, bonomial, and geometric distributions.  
It is usually used to describe prior knowledge concerning the probability of success of an event.

# Dirichlet distribution

$$X \sim \text{Dir}(\alpha)$$

Parameters:
$\alpha \in \mathbb{R}^K$, $K \geq 2$, $\alpha_k > 0$

Input: $x \in \mathbb{R}^K$, with $x_k \in [0,1]$, and $\sum_{k=1}^Kx_k=1$

$$\text{PDF: } \frac{1}{B(\alpha)} \prod_{i=1}^K x_i^{\alpha_i-1}$$

$$\text{where } B(\alpha) = \frac{\prod_{i=1}^K \Gamma(\alpha_i)}{\Gamma(\sum_{i=1}^K\alpha_i)}$$

$$E[X_i] = \frac{\alpha_i}{\sum_{k=1}^K \alpha_k}$$
$$\text{Var}(X_i) = \frac{\alpha_i(\alpha_0 - \alpha_i)}{\alpha_0^2(\alpha_0 + 1)}$$
$$\text{where } \alpha_0 = \sum_{i=1}^K \alpha_i$$  

The dirichlet distribution is a multivariate generalization of the beta distribution.  
It's the conjugate prior probability distribution of the categorical and polynomial distribution.